In [5]:
import pandas as pd

df = pd.read_csv("data/loan_data_with_predicted_categories.csv")

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "mps")

Device set to use mps


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
classifier("I love this!")

[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764600355178118},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [24]:
df['Text'][4]

'I need a loan to start a small business.'

In [25]:
classifier(df['Text'][4])

[[{'label': 'fear', 'score': 0.5070176720619202},
  {'label': 'joy', 'score': 0.290171355009079},
  {'label': 'sadness', 'score': 0.13195233047008514},
  {'label': 'anger', 'score': 0.029947245493531227},
  {'label': 'neutral', 'score': 0.028335804119706154},
  {'label': 'disgust', 'score': 0.006313301622867584},
  {'label': 'surprise', 'score': 0.006262229755520821}]]

In [28]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]

# 创建新的列来存储情绪分析结果
for label in emotion_labels:
    df[label] = np.nan  # 初始化为 NaN

# 循环遍历每一行数据，进行情绪分析
for index, row in df.iterrows():
    try:
        text = row['Text']  # 提取文本内容
        result = classifier(text)  # 进行情绪分析

        # 创建一个字典来存储当前行的情绪分析结果
        emotion_scores = {item['label']: item['score'] for item in result[0]}

        # 将结果按照 emotion_labels 的顺序存储到 DataFrame 中
        for label in emotion_labels:
            df.loc[index, label] = emotion_scores.get(label, np.nan)

    except IndexError:
        print(f"IndexError at index {index}. Skipping row.")
        continue # 跳过当前行
    except Exception as e:
        print(f"Error at index {index}: {e}. Skipping row.")
        continue # 跳过当前行


In [29]:
df.head(5)

,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status,Approval,id,tagged_text,predicted_category,anger,disgust,fear,joy,sadness,surprise,neutral
0,I need a loan to pay for an international vaca...,26556,581,8314,79.26,employed,Rejected,0,0 I need a loan to pay for an international va...,Travel,0.032620,0.009153,0.066997,0.023172,0.834439,0.003592,0.030028
1,I want to make home improvements like installi...,197392,389,111604,22.14,employed,Rejected,1,1 I want to make home improvements like instal...,Other,0.028483,0.019443,0.011550,0.083611,0.383042,0.026668,0.447203
2,"I need a loan for home renovation, including a...",44561,523,34118,45.44,employed,Rejected,2,"2 I need a loan for home renovation, including...",Renovation,0.026961,0.032717,0.055367,0.033183,0.299088,0.033726,0.518959
3,I need funds to buy new furniture and applianc...,190363,729,118757,10.22,unemployed,Rejected,3,3 I need funds to buy new furniture and applia...,House Purchase,0.032589,0.014520,0.030510,0.141397,0.558331,0.019454,0.203200
4,I need a loan to start a small business.,61853,732,19210,44.13,employed,Approved,4,4 I need a loan to start a small business.,Other,0.029947,0.006313,0.507018,0.290171,0.131952,0.006262,0.028336


In [30]:
# 保存结果到 CSV 文件
df.to_csv("data/loan_data_with_sentiment_analysis.csv", index=False)